<a href="https://colab.research.google.com/github/mikohuhu/q-learning-taxi-v3-MMAI845/blob/JAY/finalized_taxiV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import gym
import numpy as np
import time
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [15]:
# Initialize Taxi-v3 environment
env = gym.make('Taxi-v3')

In [17]:
#The first code implements a sampling method where actions are randomly selected at each timestep without any learning involved.
#This is akin to a brute-force approach, where actions are chosen without regard to the state or any learned policies.

def evaluate_sampling_method(num_episodes=100):
    penalties = []
    timesteps = []
    rewards = []

    for _ in range(num_episodes):
        state = env.reset()
        episode_penalties = 0
        episode_timesteps = 0
        episode_rewards = 0

        done = False
        while not done:
            # Sample a random action
            action = env.action_space.sample()

            # Perform the action
            next_state, reward, done, _ = env.step(action)

            # Update episode statistics
            episode_penalties += 1 if reward == -10 else 0
            episode_timesteps += 1
            episode_rewards += reward

            state = next_state

        penalties.append(episode_penalties)
        timesteps.append(episode_timesteps)
        rewards.append(episode_rewards)

    # Calculate metrics
    avg_penalties = np.mean(penalties)
    avg_timesteps = np.mean(timesteps)
    avg_rewards = np.mean(rewards)

    return avg_penalties, avg_timesteps, avg_rewards

# Perform the experiment and measure the execution time
start_time = time.time()
avg_penalties, avg_timesteps, avg_rewards = evaluate_sampling_method()
end_time = time.time()
execution_time = end_time - start_time

# Print metrics
print("Experiment 1: Sampling Method")
print("Average number of penalties per episode:", avg_penalties)
print("Average number of timesteps per trip:", avg_timesteps)
print("Average rewards per move:", avg_rewards)
print("Total time to execute:", execution_time, "seconds")

Experiment 1: Sampling Method
Average number of penalties per episode: 64.2
Average number of timesteps per trip: 195.88
Average rewards per move: -772.42
Total time to execute: 0.8409316539764404 seconds


In [18]:
env = gym.make('Taxi-v3')

def q_learning(env, num_episodes=100000, alpha=0.1, gamma=0.6, epsilon=0.1):
    Q = np.zeros((env.observation_space.n, env.action_space.n))

    total_penalties = 0
    total_timesteps = 0
    total_rewards = 0

    start_time = time.time()  # Start time

    for episode in range(num_episodes):
        state = env.reset()
        episode_penalties = 0
        episode_timesteps = 0
        episode_rewards = 0

        done = False
        while not done:
            # Epsilon-greedy action selection
            if np.random.uniform(0, 1) < epsilon:
                action = env.action_space.sample()  # Exploration
            else:
                action = np.argmax(Q[state])  # Exploitation

            next_state, reward, done, _ = env.step(action)

            # Q-Learning update
            best_next_action = np.argmax(Q[next_state])
            Q[state, action] += alpha * (reward + gamma * Q[next_state, best_next_action] - Q[state, action])

            # Update episode statistics
            episode_penalties += 1 if reward == -10 else 0
            episode_timesteps += 1
            episode_rewards += reward

            state = next_state

        total_penalties += episode_penalties
        total_timesteps += episode_timesteps
        total_rewards += episode_rewards

    # Calculate metrics
    avg_penalties = total_penalties / num_episodes
    avg_timesteps = total_timesteps / num_episodes
    avg_rewards = total_rewards / (total_timesteps - total_penalties)  # Average rewards per move

    end_time = time.time()  # End time
    execution_time = end_time - start_time  # Calculate execution time

    return avg_penalties, avg_timesteps, avg_rewards, execution_time

# Perform Q-Learning experiment
avg_penalties, avg_timesteps, avg_rewards, execution_time = q_learning(env)

# Print metrics
print("Experiment 2: Q-Learning")
print("Average number of penalties per episode:", avg_penalties)
print("Average number of timesteps per trip:", avg_timesteps)
print("Average rewards per move:", avg_rewards)
print("Total time to train:", execution_time, "seconds")

Experiment 2: Q-Learning
Average number of penalties per episode: 0.48125
Average number of timesteps per trip: 15.89639
Average rewards per move: 0.046793606804738715
Total time to train: 69.6101667881012 seconds


In [19]:
def sarsa(env, num_episodes=100000, alpha=0.2, gamma=0.8, epsilon=0.1):
    Q = np.zeros((env.observation_space.n, env.action_space.n))

    total_penalties = 0
    total_timesteps = 0
    total_rewards = 0

    start_time = time.time()  # Start time

    for episode in range(num_episodes):
        state = env.reset()
        action = epsilon_greedy_policy(Q, state, epsilon)

        episode_penalties = 0
        episode_timesteps = 0
        episode_rewards = 0

        done = False
        while not done:
            next_state, reward, done, _ = env.step(action)
            next_action = epsilon_greedy_policy(Q, next_state, epsilon)

            # SARSA update
            Q[state, action] += alpha * (reward + gamma * Q[next_state, next_action] - Q[state, action])

            # Update episode statistics
            episode_penalties += 1 if reward == -10 else 0
            episode_timesteps += 1
            episode_rewards += reward

            state = next_state
            action = next_action

        total_penalties += episode_penalties
        total_timesteps += episode_timesteps
        total_rewards += episode_rewards

    # Calculate metrics
    avg_penalties = total_penalties / num_episodes
    avg_timesteps = total_timesteps / num_episodes
    avg_rewards = total_rewards / (total_timesteps - total_penalties)  # Average rewards per move

    end_time = time.time()  # End time
    execution_time = end_time - start_time  # Calculate execution time

    return avg_penalties, avg_timesteps, avg_rewards, execution_time

def epsilon_greedy_policy(Q, state, epsilon):
    if np.random.uniform(0, 1) < epsilon:
        return np.random.choice(env.action_space.n)  # Exploration
    else:
        return np.argmax(Q[state])  # Exploitation

# Perform SARSA experiment
avg_penalties, avg_timesteps, avg_rewards, execution_time = sarsa(env)

# Print metrics
print("Experiment 3: SARSA")
print("Average number of penalties per episode:", avg_penalties)
print("Average number of timesteps per trip:", avg_timesteps)
print("Average rewards per move:", avg_rewards)
print("Total time to train:", execution_time, "seconds")

Experiment 3: SARSA
Average number of penalties per episode: 0.90008
Average number of timesteps per trip: 28.31204
Average rewards per move: -0.5942176334709375
Total time to train: 120.70162725448608 seconds


In [28]:
import numpy as np
import time
from keras.optimizers import Adam
import gym
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape
from collections import deque
import random
from tqdm import tqdm  # Importing tqdm for progress bar

# Initialize the environment with the new step API
env_taxi = gym.make("Taxi-v3", new_step_api=True).env

class TaxiAgent:
    def __init__(self, env, optimizer):
        self._state_size = env.observation_space.n
        self._action_size = env.action_space.n
        self._optimizer = optimizer
        self.experience_replay_memory = deque(maxlen=2000)
        self.discount = 0.6
        self.exploration = 0.1
        self.q_network = self._build_compile_model()
        self.target_network = self._build_compile_model()
        self.align_both_model()

    def gather(self, state, action, reward, next_state, terminated):
        self.experience_replay_memory.append((state, action, reward, next_state, terminated))

    def _build_compile_model(self):
        model = Sequential()
        model.add(Embedding(self._state_size, 10, input_length=1))
        model.add(Reshape((10,)))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(50, activation='relu'))
        model.add(Dense(self._action_size, activation='linear'))
        model.compile(loss='mse', optimizer=self._optimizer)
        return model

    def align_both_model(self):
        self.target_network.set_weights(self.q_network.get_weights())

    def act(self, state):
        if np.random.rand() <= self.exploration:
            return env_taxi.action_space.sample()
        q_values = self.q_network.predict(state, verbose=0)
        return np.argmax(q_values[0])

    def retrain(self, batch_size):
        minibatch = random.sample(self.experience_replay_memory, batch_size)
        for state, action, reward, next_state, terminated in minibatch:
            target = self.q_network.predict(state, verbose=0)
            if terminated:
                target[0][action] = reward
            else:
                t = self.target_network.predict(next_state, verbose=0)
                target[0][action] = reward + self.discount * np.amax(t)
            self.q_network.fit(state, target, epochs=1, verbose=0)  # Reduce the epochs

def deep_q_learning(env, num_training_episodes=100, num_evaluation_episodes=100, alpha=0.01, gamma=0.6, epsilon=0.1, batch_size=32, timesteps_per_episode=40, epochs=4):
    # Training phase
    start_time = time.time()  # Record start time for training
    total_penalties = 0
    total_timesteps = 0
    total_rewards = 0

    for e in tqdm(range(num_training_episodes), desc="Training Episodes"):
        state = env_taxi.reset()
        state = np.reshape(state, [1, 1])
        reward = 0
        terminated = False
        timesteps = 0
        penalties = 0

        while not terminated and timesteps < timesteps_per_episode:
            action = taxi_agent.act(state)
            next_state, reward, terminated, truncated, info = env_taxi.step(action)
            next_state = np.reshape(next_state, [1, 1])
            taxi_agent.gather(state, action, reward, next_state, terminated)
            state = next_state
            timesteps += 1
            if reward == -10:
                penalties += 1

            # Debugging print statements
            print(f"Episode {e + 1}, Timestep {timesteps}: action={action}, reward={reward}, terminated={terminated}, penalties={penalties}")

            if len(taxi_agent.experience_replay_memory) > batch_size and timesteps % 10 == 0:  # Retrain less frequently
                print(f"Retraining on mini-batch at episode {e + 1}")
                taxi_agent.retrain(batch_size)

        total_penalties += penalties
        total_timesteps += timesteps
        total_rewards += reward

    end_time = time.time()  # Record end time for training
    execution_time = end_time - start_time  # Calculate time to train

    # Evaluation phase
    total_penalties = 0
    total_timesteps = 0
    total_rewards = 0
    for e in tqdm(range(num_evaluation_episodes), desc="Evaluation Episodes"):
        # Evaluation episode loop
        state = env_taxi.reset()
        state = np.reshape(state, [1, 1])
        reward = 0
        terminated = False
        timesteps = 0
        penalties = 0

        while not terminated and timesteps < timesteps_per_episode:
            action = taxi_agent.act(state)
            next_state, reward, terminated, truncated, info = env_taxi.step(action)
            next_state = np.reshape(next_state, [1, 1])
            state = next_state
            timesteps += 1
            if reward == -10:
                penalties += 1

            # Debugging print statements
            print(f"Evaluation Episode {e + 1}, Timestep {timesteps}: action={action}, reward={reward}, terminated={terminated}, penalties={penalties}")

        total_penalties += penalties
        total_timesteps += timesteps
        total_rewards += reward

    # Calculate averages for evaluation metrics
    average_penalties = total_penalties / num_evaluation_episodes
    average_timesteps = total_timesteps / num_evaluation_episodes
    average_rewards_per_move = total_rewards / (total_timesteps - total_penalties)

    # Print metrics
    print("Deep Q-Learning Method:")
    print("Average number of penalties per episode:", average_penalties)
    print("Average number of timesteps per trip:", average_timesteps)
    print("Average rewards per move:", average_rewards_per_move)
    print("Time to train:", execution_time, "seconds")

# Creating the optimizer
optimizer = Adam(learning_rate=0.01)

# Creating the TaxiAgent instance
taxi_agent = TaxiAgent(env_taxi, optimizer)

# Defining parameters for the experiment
num_training_episodes = 100
num_evaluation_episodes = 100
batch_size = 32
timesteps_per_episode = 40

# Running the Deep Q-Learning experiment
deep_q_learning(env_taxi, num_training_episodes, num_evaluation_episodes, batch_size=batch_size, timesteps_per_episode=timesteps_per_episode)


Training Episodes:   0%|          | 0/100 [00:00<?, ?it/s]

Episode 1, Timestep 1: action=1, reward=-1, terminated=False, penalties=0
Episode 1, Timestep 2: action=1, reward=-1, terminated=False, penalties=0
Episode 1, Timestep 3: action=1, reward=-1, terminated=False, penalties=0
Episode 1, Timestep 4: action=3, reward=-1, terminated=False, penalties=0
Episode 1, Timestep 5: action=3, reward=-1, terminated=False, penalties=0
Episode 1, Timestep 6: action=4, reward=-10, terminated=False, penalties=1
Episode 1, Timestep 7: action=3, reward=-1, terminated=False, penalties=1
Episode 1, Timestep 8: action=3, reward=-1, terminated=False, penalties=1
Episode 1, Timestep 9: action=3, reward=-1, terminated=False, penalties=1
Episode 1, Timestep 10: action=3, reward=-1, terminated=False, penalties=1
Episode 1, Timestep 11: action=3, reward=-1, terminated=False, penalties=1
Episode 1, Timestep 12: action=3, reward=-1, terminated=False, penalties=1
Episode 1, Timestep 13: action=3, reward=-1, terminated=False, penalties=1
Episode 1, Timestep 14: action=3,

Training Episodes:   1%|          | 1/100 [00:11<19:02, 11.54s/it]

Episode 2, Timestep 1: action=5, reward=-10, terminated=False, penalties=1
Episode 2, Timestep 2: action=5, reward=-10, terminated=False, penalties=2
Episode 2, Timestep 3: action=5, reward=-10, terminated=False, penalties=3
Episode 2, Timestep 4: action=5, reward=-10, terminated=False, penalties=4
Episode 2, Timestep 5: action=5, reward=-10, terminated=False, penalties=5
Episode 2, Timestep 6: action=5, reward=-10, terminated=False, penalties=6
Episode 2, Timestep 7: action=5, reward=-10, terminated=False, penalties=7
Episode 2, Timestep 8: action=5, reward=-10, terminated=False, penalties=8
Episode 2, Timestep 9: action=5, reward=-10, terminated=False, penalties=9
Episode 2, Timestep 10: action=5, reward=-10, terminated=False, penalties=10
Retraining on mini-batch at episode 2
Episode 2, Timestep 11: action=2, reward=-1, terminated=False, penalties=10
Episode 2, Timestep 12: action=2, reward=-1, terminated=False, penalties=10
Episode 2, Timestep 13: action=2, reward=-1, terminated=Fa

Training Episodes:   2%|▏         | 2/100 [00:48<43:05, 26.38s/it]

...

Evaluation Episode 98, Timestep 37: action=1, reward=-1, terminated=False, penalties=2
Evaluation Episode 98, Timestep 38: action=1, reward=-1, terminated=False, penalties=2
Evaluation Episode 98, Timestep 39: action=1, reward=-1, terminated=False, penalties=2
Evaluation Episode 98, Timestep 40: action=1, reward=-1, terminated=False, penalties=2
Evaluation Episode 99, Timestep 1: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 99, Timestep 2: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 99, Timestep 3: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 99, Timestep 4: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 99, Timestep 5: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 99, Timestep 6: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 99, Timestep 7: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 99, Timestep 8: action=1, rewar

Evaluation Episodes:  99%|█████████▉| 99/100 [05:11<00:03,  3.21s/it]

Evaluation Episode 99, Timestep 40: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 100, Timestep 1: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 100, Timestep 2: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 100, Timestep 3: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 100, Timestep 4: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 100, Timestep 5: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 100, Timestep 6: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 100, Timestep 7: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 100, Timestep 8: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 100, Timestep 9: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 100, Timestep 10: action=1, reward=-1, terminated=False, penalties=0
Evaluation Episode 100, Timestep 11: actio

Evaluation Episodes: 100%|██████████| 100/100 [05:14<00:00,  3.15s/it]

Evaluation Episode 100, Timestep 39: action=1, reward=-1, terminated=False, penalties=1
Evaluation Episode 100, Timestep 40: action=1, reward=-1, terminated=False, penalties=1
Deep Q-Learning Method:
Average number of penalties per episode: 1.4
Average number of timesteps per trip: 40.0
Average rewards per move: -0.03290155440414508
Time to train: 3528.5490431785583 seconds


In [36]:
sm_data = {
    "Metric": ["Average Penalties", "Average Timesteps", "Average Rewards per Move", "Training Execution Time (seconds)"],
    "Sample Method": [64.2, 195.88, -772.42, 0.8409316539764404]
}

ql_data = {
    "Metric": ["Average Penalties", "Average Timesteps", "Average Rewards per Move", "Training Execution Time (seconds)"],
    "Q-Learning": [0.48125, 15.89639, 0.046793606804738715, 69.6101667881012]
}

sar_data = {
    "Metric": ["Average Penalties", "Average Timesteps", "Average Rewards per Move", "Training Execution Time (seconds)"],
    "SARSA": [0.90008, 28.31204, -0.5942176334709375, 120.70162725448608]
}

dql_data = {
    "Metric": ["Average Penalties", "Average Timesteps", "Average Rewards per Move", "Training Execution Time (seconds)"],
    "Deep Q-Learning": [1.4, 40.0, -0.03290155440414508, 3528.5490431785583]
}

# Create DataFrames
sm_df = pd.DataFrame(sm_data)
ql_df = pd.DataFrame(ql_data)
sarsa_df = pd.DataFrame(sar_data)
dql_df = pd.DataFrame(dql_data)

# Merge DataFrames
summary_df = pd.merge(sm_df, ql_df, on="Metric")
summary_df = pd.merge(summary_df, sarsa_df, on="Metric")
summary_df = pd.merge(summary_df, dql_df, on="Metric")
summary_df = summary_df.round(2)

print("Summary DataFrame:")
summary_df

Summary DataFrame:


,Metric,Sample Method,Q-Learning,SARSA,Deep Q-Learning
0,Average Penalties,64.20,0.48,0.90,1.40
1,Average Timesteps,195.88,15.90,28.31,40.00
2,Average Rewards per Move,-772.42,0.05,-0.59,-0.03
3,Training Execution Time (seconds),0.84,69.61,120.70,3528.55
